In [9]:

import keras
import keras_tuner
import tensorflow as tf
from IPython.core.display import Image
from keras._tf_keras.keras.preprocessing import image
from keras_tuner.src import keras_tuner_export
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils, layers
import numpy as np
import os




In [10]:
# Підготовка даних (як у умові)
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test  = x_test.reshape(10000, 784).astype('float32') / 255.0

y_train = utils.to_categorical(y_train, 10)
y_test  = utils.to_categorical(y_test, 10)

print('x_train shape:', x_train.shape, 'y_train shape:', y_train.shape)


x_train shape: (60000, 784) y_train shape: (60000, 10)


In [11]:
def build_model_part4(hp):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=(x_train.shape[1],)))

    # --- Основний цикл по кількості прихованих шарів ---
    for i in range(hp.Int('num_layers', min_value=2, max_value=5)):  # 2–5 шарів
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=128, max_value=1024, step=128),
            activation=hp.Choice(f'activation_{i}', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
        ))

    # --- Вихідний шар ---
    model.add(layers.Dense(
        y_train.shape[1],
        activation='softmax'  # бо маємо багатокласову класифікацію
    ))

    # --- Оптимізатор як гіперпараметр ---
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'SGD'])

    # --- Компіляція ---
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [12]:
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
tuner_random = RandomSearch(
    build_model_part4,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='task4_tuning',
    project_name='random_search'
)

# --- Hyperband ---
tuner_hyperband = Hyperband(
    build_model_part4,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='task4_tuning',
    project_name='hyperband_search'
)

# --- Bayesian Optimization ---
tuner_bayesian = BayesianOptimization(
    build_model_part4,
    objective='val_accuracy',
    max_trials=10,
    directory='task4_tuning',
    project_name='bayesian_search'
)

Reloading Tuner from task4_tuning\random_search\tuner0.json


In [13]:
Image('shirt.jpg', width=150, height=150)
img = image.load_img('shirt.jpg', target_size=(28, 28), color_mode="grayscale")
# Перетворюмо картинку в масив
x = image.img_to_array(img)
# Змінюємо форму масиву в плаский вектор
x = x.reshape(1, 784)
# Інвертуємо зображення
x = 255 - x
# Нормалізуємо зображення
x /= 255
classes = ['футболка', 'брюки', 'светр', 'плаття', 'пальто', 'туфлі', 'сорочка', 'кросівки', 'сумка', 'чоботи']

In [14]:
for tuner, name in zip(
    [tuner_random, tuner_hyperband, tuner_bayesian],
    ['RandomSearch', 'Hyperband', 'BayesianOptimization']
):
    print(f"\n Запускаємо {name}...")
    tuner.search(
        x_train,
        y_train,
        batch_size=256,
        epochs=20,
        validation_split=0.2
    )
    tuner.results_summary()

    models = tuner.get_best_models(num_models=3)
    for model in models:
        model.summary()
        model.evaluate(x_test, y_test)
        print()

    # Отримуємо найкращу модель
        best_model = tuner.get_best_models(num_models=1)[0]
        best_model.save(f'task4-best-{name}.keras')

        prediction = best_model.predict(x)
        prediction = np.argmax(prediction)
        print("Номер класу:", prediction)
        print("Назва класу:", classes[prediction])




Trial 10 Complete [00h 13m 06s]
val_accuracy: 0.8940833210945129

Best val_accuracy So Far: 0.8967499732971191
Total elapsed time: 02h 30m 34s
Results summary
Results in task4_tuning\bayesian_search
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 04 summary
Hyperparameters:
num_layers: 5
units_0: 896
activation_0: selu
units_1: 512
activation_1: relu
optimizer: adam
units_2: 1024
activation_2: sigmoid
units_3: 128
activation_3: selu
units_4: 896
activation_4: relu
Score: 0.8967499732971191

Trial 09 summary
Hyperparameters:
num_layers: 3
units_0: 384
activation_0: elu
units_1: 512
activation_1: elu
optimizer: adam
units_2: 512
activation_2: tanh
units_3: 256
activation_3: sigmoid
units_4: 128
activation_4: tanh
Score: 0.8940833210945129

Trial 05 summary
Hyperparameters:
num_layers: 2
units_0: 128
activation_0: selu
units_1: 384
activation_1: relu
optimizer: adam
units_2: 640
activation_2: relu
units_3: 256
activation_3: sigmoid
units_4: 1024
activation_4:

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 896)            │       703,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       459,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 896)            │       115,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         8,970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,943,690 (7.41 MB)

 Trainable params: 1,943,690 (7.41 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 384)            │        49,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         3,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 153,866 (601.04 KB)

 Trainable params: 153,866 (601.04 KB)

 Non-trainable params: 0 (0.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8868 - loss: 0.3445

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Номер класу: 6
Назва класу: сорочка
